<a href="https://colab.research.google.com/github/RickBrandao/faculdade/blob/main/Modelo_detec%C3%A7%C3%A3o_sentimentos.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [10]:
import tensorflow as tf
from tensorflow import keras
from keras import layers
from keras.saving import register_keras_serializable


In [11]:
@register_keras_serializable()
class PositionEmbedding(layers.Layer):
    def __init__(self, sequence_length, **kwargs):
        super().__init__(**kwargs)
        self.sequence_length = sequence_length

    def build(self, input_shape):
        self.embedding = self.add_weight(
            shape=(self.sequence_length, input_shape[-1]),
            initializer="uniform",
            trainable=True,
            name="position_embedding_weights"
        )

    def call(self, inputs):
        return inputs + self.embedding

    def get_config(self):
        config = super().get_config()
        config.update({"sequence_length": self.sequence_length})
        return config


In [12]:
model = keras.models.load_model("/content/bert_classificador_final.keras", custom_objects={"PositionEmbedding": PositionEmbedding})


In [13]:
from keras.layers import TextVectorization

# Simulação de vocabulário mínimo
vocab = ['[PAD]', '[UNK]', '[MASK]', 'this', 'movie', 'was', 'awesome', 'bad',
         'i', 'love', 'hate', 'boring', 'great', 'disaster', 'predictable',
         'emotional', 'rollercoaster', 'not', 'stop', 'watching']

# Cria vetor de texto com vocabulário e comprimento fixo
vectorize_layer = TextVectorization(
    max_tokens=30000,
    output_mode="int",
    output_sequence_length=256
)
vectorize_layer.set_vocabulary(vocab)



In [14]:
# Frases de teste
frases = [
    "this movie was awesome",
    "i hate this boring movie",
    "i love this great film",
    "this movie was bad"
]

# Vetoriza os textos
vetorizado = vectorize_layer(frases)

# Faz predições
preds = model.predict(vetorizado)

# Mostra resultados
for frase, prob in zip(frases, preds):
    sentimento = "Positivo" if prob[0] > 0.5 else "Negativo"
    print(f"'{frase}' → {sentimento} ({prob[0]:.2f})")


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 268ms/step
'this movie was awesome' → Negativo (0.46)
'i hate this boring movie' → Positivo (0.63)
'i love this great film' → Positivo (0.62)
'this movie was bad' → Negativo (0.34)


In [15]:
# Frases de teste – mais sutis ou ambíguas
frases = [
    "the acting was decent but the plot was weak",
    "i couldn't stop watching it",
    "i expected more from this movie",
    "not bad, but not great either"
]

# Vetoriza os textos
vetorizado = vectorize_layer(frases)

# Faz predições
preds = model.predict(vetorizado)

# Mostra resultados
for frase, prob in zip(frases, preds):
    sentimento = "Positivo" if prob[0] > 0.5 else "Negativo"
    print(f"'{frase}' → {sentimento} ({prob[0]:.2f})")


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 101ms/step
'the acting was decent but the plot was weak' → Positivo (0.57)
'i couldn't stop watching it' → Positivo (0.81)
'i expected more from this movie' → Positivo (0.66)
'not bad, but not great either' → Negativo (0.47)


In [16]:
# Frases de teste – elogios ou críticas diretas
frases = [
    "the best movie i've ever seen",
    "this film was a total disaster",
    "an emotional rollercoaster from start to finish",
    "completely boring and predictable"
]

# Vetoriza os textos
vetorizado = vectorize_layer(frases)

# Faz predições
preds = model.predict(vetorizado)

# Mostra resultados
for frase, prob in zip(frases, preds):
    sentimento = "Positivo" if prob[0] > 0.5 else "Negativo"
    print(f"'{frase}' → {sentimento} ({prob[0]:.2f})")


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 59ms/step
'the best movie i've ever seen' → Positivo (0.62)
'this film was a total disaster' → Positivo (0.62)
'an emotional rollercoaster from start to finish' → Positivo (0.69)
'completely boring and predictable' → Positivo (0.80)
